<a href="https://colab.research.google.com/github/ReaganJHarris/ET_Retrieval/blob/supplement-code/Upload_ARD_ET_maps_20210331.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#### Search **check_replace_this_var** in the notebook and correct the values  to create folder/imageCollection structures that represent your year and landsat. 

*   gDrv_CV_RootPath
*   gDrv_ARD_RootPath
*   TgtYear
*   TgtLandsat
*   DataRootFolder
*   gcs_bucket_name
*   gee_ARD_path

#### For debugging, may use the values in the "### DEBUG ###" section 

In [ ]:
import os
import sys

from urllib.parse import urlparse


In [ ]:
#@title install earthengine python API

##reference: https://developers.google.com/earth-engine/python_install_manual
#!pip install google-api-python-client
#!python -c "from oauth2client import crypt"
#!openssl version
!pip install 'pyOpenSSL>=0.11'
!pip install earthengine-api


     |████████████████████████████████| 61kB 3.6MB/s 
     |████████████████████████████████| 3.2MB 28.8MB/s 


In [ ]:
#@title set up authentication credentials (earthengine)
!earthengine authenticate

# test 1: should not show any error message with the following command
#!python -c "import ee; ee.Initialize()"
# Import the Earth Engine Python Package
import ee
# Initialize the Earth Engine object, using the authentication credentials.
ee.Initialize()

# test 2: should print the metadata of the test image
# Print the information for an image asset.
image = ee.Image('srtm90_v4')
print(image.getInfo())

Instructions for updating:
non-resource variables are not supported in the long term
To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://accounts.google.com/o/oauth2/auth?client_id=517222506229-vsmmajv00ul0bs7p89v5m89qs8eb9359.apps.googleusercontent.com&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fearthengine+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdevstorage.full_control&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&response_type=code&code_challenge=9UMv2w8rvFgNiW5LfHliPDFMdIxvQZG5M4yyRsq0Rbs&code_challenge_method=S256

The authorization workflow will generate a code, which you should paste in the box below. 
Enter verification code: 4/1AX4XfWjy2gY0fcpo_cGTajM4thKKBZRYyyMMUO7hfyH-M8B_4n7F0IsHoMg

Successfully saved authorization token.
{'type': 'Image', 'bands': [{'id': 'elevation', 'data_type': {'type': 'PixelType', '

In [ ]:
#@title install GDAL

#!python --version

#!apt-get install gdal-bin
#!apt-get install python3-gdal
#!apt-get install python3-numpy



In [ ]:
#@title mount ARD data folder on Google Drive
from google.colab import drive
drive.mount('/gdrive', force_remount=True)

Mounted at /gdrive


In [ ]:
# Note: change gDrv_CV_RootPath to '/gdrive/Shared with me'?
### check_replace_this_var ###
gDrv_CV_RootPath = '/gdrive/MyDrive'
### check_replace_this_var ###
gDrv_ARD_RootPath = os.path.join(gDrv_CV_RootPath, 'ARD_ET')
### DEBUG ###
#gDrv_ARD_RootPath = os.path.join(gDrv_CV_RootPath, 'ARD_ET', 'Temp')
#############
print(gDrv_ARD_RootPath)

/gdrive/MyDrive/ARD_ET


In [ ]:
## ARD ET Layers definition: 
## https://www.usgs.gov/core-science-systems/nli/landsat/landsat-provisional-actual-evapotranspiration
# Actual Evapotranspiration (ETa): Provides a per-pixel estimate of daily water 
#    transfer from the Earth's surface to the atmosphere in units of water depth
#    in millimeters (mm).
# ET fraction (ETf): Represents unitless fraction of ETr, nominally varying 
#    between 0 and 1 (in SSEBop model the maximum ET fraction is 1.05). 
#    This can be used in combination with user provided reference ET (ETr) 
#    to create a more accurate ETa which takes into account local weather 
#    conditions.
# Percent Uncertainty: Provides an estimate of percent uncertainty using 
#    the input ST uncertainty and the psychrometric constant.
# ET Uncertainty (ETUN): Provides ET product uncertainty in units of water
#    depth (mm) using the Percent Uncertainty value and ETr auxiliary data.
# ET Quality Assessment (ETQA): Provides high-level information about 
#    the confidence of the ETf calculated.
# Pixel Quality Assessment (pixel_qa): The bit combinations that define 
#    certain quality conditions. More information about Pixel Quality Assessment
#    can be found in the Landsat 8 Land Surface Reflectance Code (LaSRC) 
#    Product Guide and the Landsat 4-7 Surface Reflectance Product Guide 


In [ ]:
import os
import re
import subprocess
from osgeo import gdal
import xml.etree.ElementTree as ET
import datetime
import platform
import copy
#import glob
#from collections import defaultdict

class AttrDict(dict):
    __getattr__ = dict.__getitem__
    __setattr__ = dict.__setitem__

class Landsat_ARD_LST_preprocess():
    
    def __init__(self, 
                 DataRootFolder, 
                 ARDTileSearchText = r'L[CTE].*-SC.*', # r'LC.*-SC.*',
                 TgtLayers = ['eta', 'pixel_qa', 'etun', 'etf'], #['ST, PIXELQA', 'STQA']
                 ExcludeLayers = ['etqa', 'pctun']
                ):
        
        self.DataRootFolder = DataRootFolder
        self.DataWorkingFolder = os.path.join(self.DataRootFolder, 'Working')
        self.check_and_create_file_path(self.DataWorkingFolder)
        
        self.ARDTileSearchText = ARDTileSearchText
        if (type(TgtLayers) == list):
            self.TgtLayers = TgtLayers
        else:
            self.TgtLayers = [TgtLayers]
        self.ExcludeLayers = ExcludeLayers #['etqa', 'pctun']
        
        self.ARDTilePrefixArr, self.gz_FSNs = self.getARDTilePrefixArray()
        
        self.number_threads = 12
        
        if (platform.system() == 'Linux'):
            self.gsutil_exe_name = "gsutil"
            self.stdout_sep_str = '\n'
        elif (platform.system() == 'Windows'):
            self.gsutil_exe_name = "gsutil.cmd"
            self.stdout_sep_str = '\r\n'
            
        
    # get the unique ids of the ARD ET scenes
    def getARDTilePrefixArray(self):
        FP = self.DataRootFolder
        
        #FSNs_found = self.glob_re(self.ARDTileSearchText + r'.*\.tar', os.listdir(FP))
        FSNs_found = self.glob_re(self.ARDTileSearchText + r'.*\.gz', os.listdir(FP))

        FSNs_found = sorted(FSNs_found) # acsd.
        
        #print(re.split(self.ARDTileSearchText, FSNs_found[0], 1), FSNs_found[0])
        
        #
        ARDTilePrefixArr = [
            re.split(r'(^'+self.ARDTileSearchText+r')', iFSN)[1] for iFSN in FSNs_found
            #os.path.splitext(iFSN)[0] for iFSN in FSNs_found
        ]

        # get the first tif file name in the gz file and extract the Prefix string
        ARDTilePrefixArr = []
        for ii, iFSN in enumerate(FSNs_found):
            iFWN = os.path.join(self.DataRootFolder, iFSN)
            %env CURRENT_FWN=$iFWN
            fst_tif_FSN = !tar tzf $CURRENT_FWN | \
                awk -F/ '$NF ~ /\.tif$/ {print $NF ; exit }'
            if (len(fst_tif_FSN) == 1 and \
                ~str(fst_tif_FSN[0]).startswith('tar')):
                fst_tif_prefix = re.split(r'(^.*_T1'+r')', fst_tif_FSN[0])[1]
                ARDTilePrefixArr.append(fst_tif_prefix)
            else:
                raise ValueError('Error: ' + iFWN + 
                    ' doesn\'t contain tif file(s). Cannot tar!')
        
        ## debug ##
        for iPrefix, iFSN in zip(ARDTilePrefixArr, FSNs_found):
            print(iPrefix, iFSN)
        ###########
            
        return ARDTilePrefixArr, FSNs_found
    
    #--------------------------------------#
    ## preprocess downloaded ARD tiles
    #--------------------------------------#
    def extract_target_layers(self, 
                              DataRootFolder,
                              cur_gz_FSN, 
                              output_folder,
                              ExcludeLayers=['etqa', 'pctun']):
        
        exclude_args_arr = []
        for iExcludeLyr in ExcludeLayers:
            exclude_args_arr.append('--exclude')
            exclude_args_arr.append('*_' + iExcludeLyr + '.tif')
        #print(exclude_args_arr)

        #output_folder = os.path.join(DataRootFolder, 'RawTif')
        #print(output_folder)

        # # create the output folder if it doesn't exists
        # # implemented outside
        # # !mkdir -p /gdrive/MyDrive/ColoradoView/ARD_ET/Temp/RawTif
        # stdout_cr_fp = subprocess.check_output(
        #     ['mkdir', '-p', output_folder])
        
        # extract relevant tif layers
        # !tar -xvf /gdrive/MyDrive/ColoradoView/ARD_ET/Temp/LC080330322018011101T1-SC20210225173615.tar.gz \
        #      -C /gdrive/MyDrive/ColoradoView/ARD_ET/Temp/RawTif \
        #      --exclude='*_etqa.tif' --exclude='*_pctun.tif'
        cmd_str_pieces = ["tar", "-xvf", 
            os.path.join(DataRootFolder, cur_gz_FSN),
            "-C", 
            output_folder,
            ]
        cmd_str_pieces.extend(exclude_args_arr)
        print(cmd_str_pieces)

        stdout = subprocess.check_output(
            cmd_str_pieces#,
            #cwd=DataWorkingFolder
        )

        extracted_FSNs = stdout.decode("utf-8").split(sep="\n")
            
        return extracted_FSNs



    def batch_extract_target_layers(self):
        
        #for iATP in self.ARDTilePrefixArr:
        for i_gz_FSN in self.gz_FSNs:
            print("unzip {}".format(i_gz_FSN))
            res_extracted_FSNs = self.extract_target_layers(
                self.DataRootFolder,
                i_gz_FSN, 
                self.DataWorkingFolder,
                ExcludeLayers=self.ExcludeLayers #['etqa', 'pctun']
                )
        
    @staticmethod
    def convert_to_Int16_geoTiff(in_TBConv_Tiff_FWN, out_Converted_Tiff_FWN, number_threads=1):
        # convert Data Type from UInt16 to Int16 for the PIXELQA layer
        translateoptions = gdal.TranslateOptions(
            gdal.ParseCommandLine(
                "-ot Int16 -of Gtiff -co COMPRESS=LZW -co NUM_THREADS={}".format(number_threads))
        )
        gdal.Translate(out_Converted_Tiff_FWN, in_TBConv_Tiff_FWN, options=translateoptions)
        
    def batch_convert_PIXELQA_to_Int16(self):
        
        for iATP in self.ARDTilePrefixArr:
            #iTBConv_Tiff_FWN = os.path.join(self.DataWorkingFolder, "{}_PIXELQA.tif".format(iATP))
            iTBConv_Tiff_FWN = os.path.join(self.DataWorkingFolder, "{}_pixel_qa.tif".format(iATP))
            iConverted_Tiff_FWN = os.path.join(self.DataWorkingFolder, "{}_pixel_qa_int16.tif".format(iATP))
            
            print("Converting {}".format(os.path.basename(iTBConv_Tiff_FWN)))
            
            self.convert_to_Int16_geoTiff(iTBConv_Tiff_FWN, iConverted_Tiff_FWN, number_threads=self.number_threads)
            
    def stack_target_layers(self, File_Prefix, TgtLayers_Int16, number_threads=1):

        # stack 'eta', 'pixel_qa_int16', 'etun', and 'etf' layers
        out_stacked_FWN1 = os.path.join(self.DataWorkingFolder, "{}_stacked.tif".format(File_Prefix))
        out_stacked_vrt_FWN1 = os.path.join(self.DataWorkingFolder, "{}_stacked_tmp.vrt".format(File_Prefix))
        print(out_stacked_FWN1)
        
        in_layers_FWNs = [
            os.path.join(self.DataWorkingFolder, "{}_{}.tif".format(File_Prefix, iTgtLyrInt16)) \
                         for iTgtLyrInt16 in TgtLayers_Int16
            ]

        buildVRToptions = gdal.BuildVRTOptions(gdal.ParseCommandLine("-separate"))
        gdal.BuildVRT(out_stacked_vrt_FWN1, in_layers_FWNs, options=buildVRToptions)

        translateoptions = gdal.TranslateOptions(gdal.ParseCommandLine("-co COMPRESS=LZW -co NUM_THREADS={}".format(number_threads)))
        gdal.Translate(out_stacked_FWN1, out_stacked_vrt_FWN1, options=translateoptions)
        
    def batch_stack_target_layers(self, TgtLayers_Int16):
        
        for iATP in self.ARDTilePrefixArr:
            print("Stacking {}".format(iATP))
            self.stack_target_layers(iATP, TgtLayers_Int16, number_threads=self.number_threads)
    
    #--------------------------------------#
    ## GCS and GEE operations
    #--------------------------------------#
    def upload_one_stacked_tile_to_google_cloud_bucket(self, File_Prefix, gcs_bucket_name):
        
        # locate the stacked .tif file
        TBUpload_stacked_FWN1 = os.path.join(self.DataWorkingFolder, "{}_stacked.tif".format(File_Prefix))
        print("Uploading", TBUpload_stacked_FWN1)
        
        try:
            TBU_FSN1 = os.path.basename(TBUpload_stacked_FWN1)
            gcs_FWN1 = os.path.join(gcs_bucket_name, TBU_FSN1)
          
            ##'gsutil.cmd --help',
            ##'gsutil cp *V01.xml gs://test-gee-upload-ard-lst',
            stdout = subprocess.check_output(
                [self.gsutil_exe_name, "cp", TBUpload_stacked_FWN1, gcs_FWN1], #"gsutil.cmd"
                cwd=self.DataWorkingFolder
                )
            screen_output_str_arr = stdout.decode().split(self.stdout_sep_str)
            for iSO in screen_output_str_arr:
                if (len(iSO) > 0): 
                    print(iSO)
        except subprocess.CalledProcessError as e:
            print(e.output)
        
        
    def batch_upload_stacked_tiles_to_google_cloud_bucket(self, gcs_bucket_name):
        for iATP in self.ARDTilePrefixArr:
            self.upload_one_stacked_tile_to_google_cloud_bucket(iATP, gcs_bucket_name)
        
        print("Uploads finished.")
        
    
    def list_gcs_bucket_files(self, gcs_bucket_name):
        try:

            stdout = subprocess.check_output([self.gsutil_exe_name, "ls", gcs_bucket_name]) # "gsutil.cmd" gs://
            screen_output_str_arr = stdout.decode().split(self.stdout_sep_str)
            gcs_paths = []
            for iSO in screen_output_str_arr:
                if (len(iSO) > 0): 
                    print(iSO)
                    gcs_paths.append(iSO)
            return gcs_paths
        except subprocess.CalledProcessError as e:
            print(e.output)
            

    @staticmethod
    def create_GEE_ARD_Landsat8_ET_ImageCollection(gee_ARD_path):
        # create ImageCollection to hold Landsat ARD ET image tiles
        ##projects/ColoradoView/Landsat/ARD_ET/8
        cmd = ["earthengine", "create", "collection", "-p", gee_ARD_path] 
        try:
            stdout = subprocess.check_output(cmd)
        except subprocess.CalledProcessError as e:
            print(e.output)
        
    
    def batch_upload_ARD_tiles_from_google_cloud_bucket(
        self, 
        gcs_paths, 
        gee_ARD_path, 
        in_TBE_attribs, 
        attrib_name_sys_time='system:time_start'):
      
        # Upload each image from GCS
        for path in gcs_paths:
            print("uploading {} to gee {}.".format(path, gee_ARD_path))
            TBE_attribs = copy.deepcopy(in_TBE_attribs)
            FSN = path.split("/")[-1]
            name = os.path.splitext(FSN)[0]
            File_Prefix = name[:-8] # strip the "_stacked"
            if (gee_ARD_path[-1] != '/'):
                gee_ARD_path = gee_ARD_path + '/'
            Full_asset_name = gee_ARD_path + name
            #options_others = " --pyramiding_policy=mean --nodata_value -9999 "
            options_others = ["--pyramiding_policy=mean", "--nodata_value=-9999"]
            # prepare attributes
            xml_FWN = os.path.join(self.DataWorkingFolder, File_Prefix + '.xml')
            combined_attr_dict = self.parse_one_ARD_xml(xml_FWN, TBE_attribs, attrib_name_sys_time = attrib_name_sys_time)
            property_op_str_arr = []
            time_start_str = ''
            for i_k, i_v in combined_attr_dict.items():
                if (i_k == 'system:time_start'):
                  time_start_str = '--time_start={}'.format(i_v)
                  continue
                if (self.is_str_date(i_v)):
                    v_type = 'date'
                elif (self.is_number(i_v)):
                    v_type = 'number'
                else:
                    v_type = 'string'
                property_op_str_arr.append(
                    "-p({}){}={}".format(v_type, i_k, i_v)
                    )
            options_Attribs = " ".join(property_op_str_arr)
            
            #"earthengine upload image --asset_id=users/stevenf/cms/rehfeldt_1km/" + name + 
            #"  --pyramiding_policy=mean --nodata_value -500 " + path
            cmd = ["earthengine", "upload", "image"]
            cmd.extend(["--asset_id={}".format(Full_asset_name)])
            cmd.extend(property_op_str_arr)
            cmd.extend(options_others)
            cmd.extend([time_start_str])
            cmd.extend([path])
            print(cmd)
            try:
                stdout = subprocess.check_output(cmd)
                #pass
            except subprocess.CalledProcessError as e:
                print(e.output)
                
        print("Uploading to gee finished.")
        

    #--------------------------------------#
    ## xml parsing
    #--------------------------------------#
    @staticmethod
    def parse_one_ARD_tile_xml_global_attributes(xml_FWN, 
                               target_tag_arr, 
                               gee_attrib_names,
                               name_space_dict = {'ns0': 'http://espa.cr.usgs.gov/v2'} #{'ns0': 'https://landsat.usgs.gov/ard/v1'} 
                              ):
        
        ret_dict = AttrDict()
    
        cur_xml_tree = ET.parse(xml_FWN)
        cur_xml_root = cur_xml_tree.getroot()
        
        ns = name_space_dict
        
        for iTag, iGEEName in zip(target_tag_arr, gee_attrib_names):
            #("ns0:tile_metadata/"
            # "ns0:global_metadata/"
            # "ns0:projection_information")
            iTagInfo = cur_xml_root.findall(iTag, ns)
            
            ## main field text
            if (iTagInfo[0].text is not None):
                iTagInfo0_text = iTagInfo[0].text
                iTagInfo0_text = re.sub("\s", '', iTagInfo0_text)
                if (len(iTagInfo0_text) > 0):
                    #print(iGEEName, iTagInfo[0].text)
                    ret_dict[iGEEName] = iTagInfo[0].text
            
            ## attributes
            ###
            #<bounding_coordinates>
            #    <west>-110.131907577</west>
            #    <east>-108.18643133</east>
            #    <north>38.7343536882</north>
            #    <south>37.2242306639</south>
            #</bounding_coordinates>
            if (len(iTagInfo[0]) > 0):
                # sub nodes 
                for iSubE in list(iTagInfo[0].iter()):
                    Text_iSubE = iSubE.text
                    Text_iSubE = re.sub("\s", '', Text_iSubE)
                    TagName_iSubE = iSubE.tag
                    TagName_iSubE = re.sub("{" + name_space_dict.get(list(name_space_dict.keys())[0]) + "}",
                                        '',
                                        TagName_iSubE
                                       )
                    if (len(Text_iSubE) > 0):
                        iSubE_GEEName = iGEEName + '_' + TagName_iSubE
                        #print(iSubE_GEEName, Text_iSubE)
                        ret_dict[iSubE_GEEName] = Text_iSubE

            ###
            # <tile_grid h="009" v="010"/>
            iTag_items = iTagInfo[0].items()
            if (len(iTag_items) > 0):
                for jIt in iTag_items:
                    jIt_GEEName = iGEEName + '_' + jIt[0]
                    #print(jIt_GEEName, jIt[1])
                    ret_dict[jIt_GEEName] = jIt[1]
                    
        return ret_dict

    
    @staticmethod
    def parse_one_ARD_tile_xml_one_band_attributes(
        xml_FWN, 
        target_band_path, #"ns0:bands/"
        target_band_name, #"eta"
        target_band_attribs, # ['add_offset', 'data_type', 'name', 'valid_range']
        name_space_dict = {'ns0': 'http://espa.cr.usgs.gov/v2'} #{'ns0': 'https://landsat.usgs.gov/ard/v1'} 
        ):
        
        def search_matched_attrib_name(attrib_name, TBC_attrib_name_arr):
            try:
                idx = TBC_attrib_name_arr.index(attrib_name)
            except:
                idx = -1
            return idx
        
        ret_dict = AttrDict()
    
        cur_xml_tree = ET.parse(xml_FWN)
        cur_xml_root = cur_xml_tree.getroot()
        
        ns = name_space_dict
        
        ## find the node of the target band
        #("ns0:bands/")
        TagInfo_arr = cur_xml_root.findall(target_band_path, ns)
        for iTagInfo in TagInfo_arr:
            ## find target band by "name" 
            # <band product="evapotranspiration" source="toa_refl" name="eta" 
            # category="image" data_type="INT16" nlines="7831" nsamps="7701" 
            # fill_value="-9999" scale_factor="0.001000">
            iTag_items = iTagInfo.items()
            if (len(iTag_items) > 0):
                Is_iTag_target = False
                for jIt in iTag_items:
                    if (jIt[0] == 'name' and jIt[1] == target_band_name):
                        Is_iTag_target = True
                        break
                if (Is_iTag_target == False):
                    # not the target band, searching the next band
                    #print("not the target band, searching the next band")
                    continue
                
                for jIt in iTag_items:
                    jIt_idx = search_matched_attrib_name(jIt[0], target_band_attribs)
                    if (jIt_idx >= 0):
                        jIt_GEEName = target_band_name + '_' + jIt[0]
                        print(jIt_GEEName, jIt[1])
                        ret_dict[jIt_GEEName] = jIt[1]
                        del target_band_attribs[jIt_idx]
                        
            print("{} band attribs in sub elements:".format(target_band_name))
            
            if (len(iTagInfo) > 0):
                # sub nodes 
                for iSubE in list(iTagInfo.iter()):

                    TagName_iSubE = iSubE.tag
                    TagName_iSubE = re.sub("{" + name_space_dict.get(list(name_space_dict.keys())[0]) + "}",
                                        '',
                                        TagName_iSubE
                                       )
                    TagName_iSubE_idx = search_matched_attrib_name(TagName_iSubE, target_band_attribs)
                    if (TagName_iSubE_idx < 0):
                        continue
                    #print("TagName_iSubE", TagName_iSubE, isinstance(iSubE, str), type(iSubE))
                    
                    #if (not isinstance(iSubE, str)):
                    ##            
                    # <valid_range min="0.000000" max="20000.000000"/>
                    # <pixel_size x="30" y="30" units="meters"/>
                    iSubE_items = iSubE.items()
                    if (len(iSubE_items) > 0):
                        for jIt in iSubE_items:
                            jIt_GEEName = target_band_name + '_' + TagName_iSubE + '_' + jIt[0]
                            #print(jIt_GEEName, jIt[1])
                            ret_dict[jIt_GEEName] = jIt[1]
                    else:
                        ##
                        # <data_units>mm</data_units>
                        Text_iSubE = iSubE.text
                        Text_iSubE = re.sub("\s", '', Text_iSubE)
                        if (len(Text_iSubE) > 0):
                            iSubE_GEEName = target_band_name + '_' + TagName_iSubE
                            #print(iSubE_GEEName, Text_iSubE)
                            ret_dict[iSubE_GEEName] = Text_iSubE
                            
                    del target_band_attribs[TagName_iSubE_idx]
                                
                 
            # break from band loop
            break
            
        if (len(target_band_attribs) > 0):
            print("remaining target_band_attribs", target_band_attribs)
            
        return ret_dict
    
    def parse_one_ARD_xml(self, xml_FWN, TBE_attribs, attrib_name_sys_time='system:time_start'):
        ns = 'ns0'
        name_space_dict = {ns: 'http://espa.cr.usgs.gov/v2'} #{ns: 'https://landsat.usgs.gov/ard/v1'}

        #### global attributes ####
        tag_str_prefix_tile_global = "{}:global_metadata/".format(ns) #"{}:tile_metadata/{}:global_metadata/".format(ns, ns)
        global_tag_strs = TBE_attribs['global']
        global_tag_xml_path_arr = [
            "{}{}:{}".format(tag_str_prefix_tile_global, ns, iTagStr) for iTagStr in global_tag_strs
            ]        
        global_gee_attrib_names = global_tag_strs
        glb_attr_dict = self.parse_one_ARD_tile_xml_global_attributes(
            xml_FWN, 
            global_tag_xml_path_arr, 
            global_gee_attrib_names,
            name_space_dict=name_space_dict)

        print("glb_attr_dict", glb_attr_dict)
        
        #### band attributes ####
        TgtLayers_attribs = TBE_attribs['bands']
        TgtLayers_attribs_keys = list(TgtLayers_attribs.keys())
        
        ## Debug ##
        #print("TgtLayers_attribs_keys", TgtLayers_attribs_keys)
        ###########
        
        
        #tag_str_prefix_tile_band = "{}:tile_metadata/{}:bands/".format(ns, ns, ns)
        tag_str_prefix_tile_band = "{}:bands/".format(ns)
        
        attr_dict_all_bands = []
        for iBand in TgtLayers_attribs_keys:
            print("processing band [{}]".format(iBand))
                        
            target_band_path_iBand = tag_str_prefix_tile_band
            target_band_name_iBand = iBand
            #["name", "scale_factor", "add_offset", "data_type", "fill_value", "nlines", "nsamps",
            # "pixel_size", "data_units", "valid_range", "app_version"]
            target_band_attribs_iBand = TgtLayers_attribs[iBand]
            attr_dict_iBand = self.parse_one_ARD_tile_xml_one_band_attributes(
                xml_FWN, 
                target_band_path_iBand, #"ns0:bands/"
                target_band_name_iBand, #"eta"
                target_band_attribs_iBand, # ['add_offset', 'data_type', 'name', 'valid_range']
                name_space_dict = name_space_dict #{'ns0': 'https://landsat.usgs.gov/ard/v1'} 
            )
            attr_dict_all_bands.append(attr_dict_iBand)
            print("attr_dict_iBand [{}]".format(iBand), attr_dict_iBand)
        
        #### combined attribs ####
        ## {**glb_attr_dict, **ST_attr_dict, **STQA_attr_dict, **PIXELQA_attr_dict}
        combined_attr_dict_arr = [glb_attr_dict, *attr_dict_all_bands]
        combined_attr_dict = glb_attr_dict.copy()
        for iDict in attr_dict_all_bands:
            combined_attr_dict = {**combined_attr_dict, **iDict}
        
        ## change the attrib name in attrib_name_sys_time to 'system:time_start'
        combined_attr_dict['system:time_start'] = combined_attr_dict.pop(attrib_name_sys_time)
        
        print("\nCombined_attr_dict:\n")
        for cmb_k, cmb_v in combined_attr_dict.items():
            if (self.is_str_date(cmb_v)):
                v_type = 'date'
            elif (self.is_number(cmb_v)):
                v_type = 'number'
            else:
                v_type = 'str'
            print(cmb_k, cmb_v, v_type)

        return combined_attr_dict
    
    #--------------------------------------#
    ## Utilities
    #--------------------------------------#
    # search files using a regex pattern
    @staticmethod
    def glob_re(pattern, strings):
        return list(filter(re.compile(pattern).search, strings))
        #return list(filter(re.compile(pattern).match, strings))
        
    # check and create the path if not exists
    @staticmethod
    def check_and_create_file_path(in_path, in_Path_desc_str=None):
        if (in_Path_desc_str is None):
            Path_desc_str = ""
        else:
            Path_desc_str = in_Path_desc_str
        try:
            os.stat(in_path)
            print("{} Path [{}] exists.".format(Path_desc_str, in_path))
        except:
            #os.mkdir(in_path) 
            os.makedirs(in_path)
            print("{} Path [{}] was created.".format(Path_desc_str, in_path))
            
    @staticmethod
    def path_escape_space(in_path):
      return re.sub(' ', r'\ ',in_path)
      
            
    @staticmethod
    def is_str_date(in_str, date_format='%Y-%m-%d'):
        try:
            datetime.datetime.strptime(in_str, date_format)
            return True
        except ValueError:
            return False
        
    @staticmethod
    def is_number(in_str):
        try:
            float(in_str)
            return True
        except ValueError:
            pass

        #try:
        #    import unicodedata
        #    unicodedata.numeric(in_str)
        #    return True
        #except (TypeError, ValueError):
        #    pass

        return False
        

In [ ]:
#@title preprocess parameters

import re

### check_replace_this_var ###
# TgtYear = 2011 #2015
TgtCounty = '2018_Phillips' #Weld
### check_replace_this_var ###
TgtLandsat = '8'
#gDrv_ARD_RootPath_escaped = re.sub(" ", r'\ ', gDrv_ARD_RootPath)
#DataRootFolder = os.path.join(gDrv_ARD_RootPath_escaped, "{}".format(TgtYear), TgtLandsat)
### check_replace_this_var ###
# DataRootFolder = os.path.join(gDrv_ARD_RootPath, "{}".format(TgtYear), 
                              # TgtLandsat)
DataRootFolder = os.path.join(gDrv_ARD_RootPath, TgtCounty, 
                              TgtLandsat)
### DEBUG ###
#DataRootFolder = gDrv_ARD_RootPath
#############
print("DataRootFolder", DataRootFolder)


# ARDTileSearchText = r'.*C\d\d_V\d\d' #r'.*C\d\d_V\d\d_ST\.tar' #r'*C\d\d_D\d\d\.xml'
# TgtLayers = ['ST', 'PIXELQA', 'STQA']
# TgtLayers_Int16 = ['ST', 'PIXELQA_Int16', 'STQA']
ARDTileSearchText = r'L[CET].*-SC.*' # r'LC.*-SC.*'
TgtLayers = ['eta', 'pixel_qa', 'etun', 'etf'] #['ST, PIXELQA', 'STQA']
TgtLayers_Int16 = ['eta', 'pixel_qa_int16', 'etun', 'etf']
ExcludeLayers = ['etqa', 'pctun']


#gcs_bucket_root_name = r'gs://test-gee-upload-ard-lst'
gcs_bucket_root_name = r'gs://test-gee-upload-ard-et'
### check_replace_this_var ###
# gcs_bucket_name = os.path.join(gcs_bucket_root_name, str(TgtYear), TgtLandsat)
gcs_bucket_name = os.path.join(gcs_bucket_root_name, TgtCounty, TgtLandsat)
### DEBUG ###
#gcs_bucket_name = os.path.join(gcs_bucket_root_name, 'tmp')
#############

print("gcs_bucket_name", gcs_bucket_name)

#gee_ARD_path = r'projects/ColoradoView/Landsat/ARD/' + TgtLandsat
### check_replace_this_var ###
gee_ARD_path = r'projects/ColoradoView/Landsat/ARD_ET/' + TgtLandsat
### DEBUG ###
#gee_ARD_path = r'projects/ColoradoView/Landsat/ARD_ET/' + 'tmp'
#############

global_tag_strs = ['acquisition_date', 'satellite', 'instrument', 
                   'product_id', 'bounding_coordinates'
                   ] 
                   # 'ard_version', 'tile_grid', 'cloud_cover', 
                   # 'cloud_shadow', 'snow_ice', 'fill'
target_band_attribs_eta = ["name", "scale_factor",  
                          "long_name", "data_type", 
                          "fill_value", "nlines", 
                          "nsamps", "pixel_size", "data_units", 
                          "valid_range", "app_version"]
                          # "add_offset",
target_band_attribs_etun = ["name", "scale_factor", 
                            "long_name", "data_type", 
                            "fill_value", "nlines", "nsamps",
                            "pixel_size", "data_units", 
                            "valid_range", "app_version"]
target_band_attribs_pixel_qa = ["name", "data_type", 
                                "long_name", "fill_value", 
                                "nlines", "nsamps", "data_units", 
                                "pixel_size", "app_version"] 
                                # "valid_range"
                               
target_band_attribs_etf = ["name", "scale_factor", 
                           "long_name", "data_type",
                           "fill_value", "nlines", "nsamps",
                           "pixel_size", "data_units", 
                           "valid_range", "app_version"
                           ]                   
TgtLayers_attribs = {
    'eta': target_band_attribs_eta, 
    'pixel_qa': target_band_attribs_pixel_qa,
    'etun': target_band_attribs_etun,
    'etf': target_band_attribs_etf
    }
TBE_attribs = {'global': global_tag_strs, 'bands': TgtLayers_attribs}



DataRootFolder /gdrive/MyDrive/ARD_ET/2018_Phillips/8
gcs_bucket_name gs://test-gee-upload-ard-et/2018_Phillips/8


In [ ]:
#@title initialize the Landsat_ARD_LST_preprocess class
LALP_obj1 = Landsat_ARD_LST_preprocess(
  DataRootFolder,
  ARDTileSearchText=ARDTileSearchText,
  TgtLayers=TgtLayers,
  ExcludeLayers=ExcludeLayers
  )


 Path [/gdrive/MyDrive/ARD_ET/2018_Phillips/8/Working] was created.
env: CURRENT_FWN=/gdrive/MyDrive/ARD_ET/2018_Phillips/8/LC080320322018010401T1-SC20210707033825.tar.gz
env: CURRENT_FWN=/gdrive/MyDrive/ARD_ET/2018_Phillips/8/LC080320322018041001T1-SC20210707034317.tar.gz
env: CURRENT_FWN=/gdrive/MyDrive/ARD_ET/2018_Phillips/8/LC080320322018042601T1-SC20210707041440.tar.gz
env: CURRENT_FWN=/gdrive/MyDrive/ARD_ET/2018_Phillips/8/LC080320322018061301T1-SC20210707041107.tar.gz
env: CURRENT_FWN=/gdrive/MyDrive/ARD_ET/2018_Phillips/8/LC080320322018062901T1-SC20210707041708.tar.gz
env: CURRENT_FWN=/gdrive/MyDrive/ARD_ET/2018_Phillips/8/LC080320322018071501T1-SC20210707043948.tar.gz
env: CURRENT_FWN=/gdrive/MyDrive/ARD_ET/2018_Phillips/8/LC080320322018073101T1-SC20210707044010.tar.gz
env: CURRENT_FWN=/gdrive/MyDrive/ARD_ET/2018_Phillips/8/LC080320322018081601T1-SC20210707040456.tar.gz
env: CURRENT_FWN=/gdrive/MyDrive/ARD_ET/2018_Phillips/8/LC080320322018090101T1-SC20210707034518.tar.gz
env: 

In [ ]:
!ls /gdrive/MyDrive/ARD_ET/Moffat/5/Working

**local preprocess: extract, convert DType, stack**

In [ ]:
#@title batch extract target layers
LALP_obj1.batch_extract_target_layers()

unzip LC080320322018010401T1-SC20210707033825.tar.gz
['tar', '-xvf', '/gdrive/MyDrive/ARD_ET/2018_Phillips/8/LC080320322018010401T1-SC20210707033825.tar.gz', '-C', '/gdrive/MyDrive/ARD_ET/2018_Phillips/8/Working', '--exclude', '*_etqa.tif', '--exclude', '*_pctun.tif']
unzip LC080320322018041001T1-SC20210707034317.tar.gz
['tar', '-xvf', '/gdrive/MyDrive/ARD_ET/2018_Phillips/8/LC080320322018041001T1-SC20210707034317.tar.gz', '-C', '/gdrive/MyDrive/ARD_ET/2018_Phillips/8/Working', '--exclude', '*_etqa.tif', '--exclude', '*_pctun.tif']
unzip LC080320322018042601T1-SC20210707041440.tar.gz
['tar', '-xvf', '/gdrive/MyDrive/ARD_ET/2018_Phillips/8/LC080320322018042601T1-SC20210707041440.tar.gz', '-C', '/gdrive/MyDrive/ARD_ET/2018_Phillips/8/Working', '--exclude', '*_etqa.tif', '--exclude', '*_pctun.tif']
unzip LC080320322018061301T1-SC20210707041107.tar.gz
['tar', '-xvf', '/gdrive/MyDrive/ARD_ET/2018_Phillips/8/LC080320322018061301T1-SC20210707041107.tar.gz', '-C', '/gdrive/MyDrive/ARD_ET/2018_

In [ ]:
#@title batch convert PIXELQA to Int16 type 
## because ST and STQA have the Int16 type, and
## layers to be stacked must have the same data type
LALP_obj1.batch_convert_PIXELQA_to_Int16()

Converting LC08_L1TP_032032_20180104_20180118_01_T1_pixel_qa.tif
Converting LC08_L1TP_032032_20180410_20180417_01_T1_pixel_qa.tif
Converting LC08_L1TP_032032_20180426_20180502_01_T1_pixel_qa.tif
Converting LC08_L1TP_032032_20180613_20180703_01_T1_pixel_qa.tif
Converting LC08_L1TP_032032_20180629_20180716_01_T1_pixel_qa.tif
Converting LC08_L1TP_032032_20180715_20180730_01_T1_pixel_qa.tif
Converting LC08_L1TP_032032_20180731_20180814_01_T1_pixel_qa.tif
Converting LC08_L1TP_032032_20180816_20180829_01_T1_pixel_qa.tif
Converting LC08_L1TP_032032_20180901_20180912_01_T1_pixel_qa.tif
Converting LC08_L1TP_032032_20180917_20180928_01_T1_pixel_qa.tif
Converting LC08_L1TP_032032_20181003_20181010_01_T1_pixel_qa.tif
Converting LC08_L1TP_032032_20181019_20181031_01_T1_pixel_qa.tif
Converting LC08_L1TP_032032_20181104_20181115_01_T1_pixel_qa.tif
Converting LC08_L1TP_032032_20181120_20181129_01_T1_pixel_qa.tif
Converting LC08_L1TP_032033_20180104_20180118_01_T1_pixel_qa.tif
Converting LC08_L1TP_0320

In [ ]:
# !ls -l /gdrive/MyDrive/ARD_ET/2013/8/Working/LC08_L1TP_035034_20130620_20170309_01_T1_pixel_qa*
# !ls -a --help
import numpy
from PIL import Image
# im = Image.open('/gdrive/MyDrive/ARD_ET/2018/8/Working/LC08_L1TP_036032_20181031_20181115_01_T1_eta.tif')
im = Image.open('/gdrive/MyDrive/ARD_ET/2013/8/Working/LC08_L1TP_035034_20130620_20170309_01_T1_pixel_qa.tif')
im.show()
# print(numpy.array(im))
# print(numpy.mean(im))
im = numpy.array(im)
im_mask = numpy.greater_equal(im, 0)
# print(numpy.mean(im[im_mask]))

In [ ]:
from  xarray import open_rasterio

In [ ]:
!pip install -U rasterio

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
qa = open_rasterio('/gdrive/MyDrive/ARD_ET/2013/8/Working/LC08_L1TP_035034_20130620_20170309_01_T1_pixel_qa.tif')

fig = plt.figure(figsize=(16,16))
ax = fig.add_subplot(111)
ax.imshow(qa.variable.data[0])
plt.show()

In [ ]:
#@title batch stack target layers
LALP_obj1.batch_stack_target_layers(TgtLayers_Int16)

Stacking LC08_L1TP_032032_20180104_20180118_01_T1
/gdrive/MyDrive/ARD_ET/2018_Phillips/8/Working/LC08_L1TP_032032_20180104_20180118_01_T1_stacked.tif
Stacking LC08_L1TP_032032_20180410_20180417_01_T1
/gdrive/MyDrive/ARD_ET/2018_Phillips/8/Working/LC08_L1TP_032032_20180410_20180417_01_T1_stacked.tif
Stacking LC08_L1TP_032032_20180426_20180502_01_T1
/gdrive/MyDrive/ARD_ET/2018_Phillips/8/Working/LC08_L1TP_032032_20180426_20180502_01_T1_stacked.tif
Stacking LC08_L1TP_032032_20180613_20180703_01_T1
/gdrive/MyDrive/ARD_ET/2018_Phillips/8/Working/LC08_L1TP_032032_20180613_20180703_01_T1_stacked.tif
Stacking LC08_L1TP_032032_20180629_20180716_01_T1
/gdrive/MyDrive/ARD_ET/2018_Phillips/8/Working/LC08_L1TP_032032_20180629_20180716_01_T1_stacked.tif
Stacking LC08_L1TP_032032_20180715_20180730_01_T1
/gdrive/MyDrive/ARD_ET/2018_Phillips/8/Working/LC08_L1TP_032032_20180715_20180730_01_T1_stacked.tif
Stacking LC08_L1TP_032032_20180731_20180814_01_T1
/gdrive/MyDrive/ARD_ET/2018_Phillips/8/Working/LC0

**google cloud storage and google earth engine operations**



In [ ]:
#@title initialize google cloud permission
!gcloud init

Welcome! This command will take you through the configuration of gcloud.

Settings from your current configuration [default] are:
component_manager:
  disable_update_check: 'True'
compute:
  gce_metadata_read_timeout_sec: '0'

Pick configuration to use:
 [1] Re-initialize this configuration [default] with new settings 
 [2] Create a new configuration
Please enter your numeric choice:  1

Your current configuration has been set to: [default]

You can skip diagnostics next time by using the following flag:
  gcloud init --skip-diagnostics

Network diagnostic detects and fixes local network connection issues.
Reachability Check passed.
Network diagnostic passed (1/1 checks passed).

You must log in to continue. Would you like to log in (Y/n)?  Y

Go to the following link in your browser:

    https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=openid+https%3A%2F%2Fwww.googleapi

In [ ]:
#@title create the bucket folder if not exists
print("gcs_bucket_name: ", gcs_bucket_name)
gcs_bucket_hostname = urlparse(gcs_bucket_name).scheme +  '://' + \
                      urlparse(gcs_bucket_name).hostname
print(gcs_bucket_hostname)

# get the current region of the project 
# (if not set gsutil mb will create a multi-region bucket)
res_reg = !gcloud config get-value compute/region
cur_region = res_reg[0]
print(cur_region)

# test whether the bucket root path exists
test_b_exist = !gsutil ls -b {gcs_bucket_hostname}
##print(str(test_b_exist[0]).startswith(gcs_bucket_hostname))

# only create bucket if the bucket root path doesn't exist
if not(str(test_b_exist[0]).startswith(gcs_bucket_hostname)):
    print("Bucket root path doesn't exist. ")
    !gsutil mb -l {cur_region} {gcs_bucket_hostname}
else:
    print("Bucket root path {} already exists.".format(gcs_bucket_hostname))



gcs_bucket_name:  gs://test-gee-upload-ard-et/2018_Phillips/8
gs://test-gee-upload-ard-et
(unset)
Bucket root path doesn't exist. 
/bin/bash: -c: line 0: syntax error near unexpected token `('
/bin/bash: -c: line 0: `gsutil mb -l (unset) gs://test-gee-upload-ard-et'


In [ ]:
#@title batch upload stacked tiles to google cloud storage 
LALP_obj1.batch_upload_stacked_tiles_to_google_cloud_bucket(gcs_bucket_name)

Uploading /gdrive/MyDrive/ARD_ET/2018_Phillips/8/Working/LC08_L1TP_032032_20180104_20180118_01_T1_stacked.tif
Uploading /gdrive/MyDrive/ARD_ET/2018_Phillips/8/Working/LC08_L1TP_032032_20180410_20180417_01_T1_stacked.tif
Uploading /gdrive/MyDrive/ARD_ET/2018_Phillips/8/Working/LC08_L1TP_032032_20180426_20180502_01_T1_stacked.tif
Uploading /gdrive/MyDrive/ARD_ET/2018_Phillips/8/Working/LC08_L1TP_032032_20180613_20180703_01_T1_stacked.tif
Uploading /gdrive/MyDrive/ARD_ET/2018_Phillips/8/Working/LC08_L1TP_032032_20180629_20180716_01_T1_stacked.tif
Uploading /gdrive/MyDrive/ARD_ET/2018_Phillips/8/Working/LC08_L1TP_032032_20180715_20180730_01_T1_stacked.tif
Uploading /gdrive/MyDrive/ARD_ET/2018_Phillips/8/Working/LC08_L1TP_032032_20180731_20180814_01_T1_stacked.tif
Uploading /gdrive/MyDrive/ARD_ET/2018_Phillips/8/Working/LC08_L1TP_032032_20180816_20180829_01_T1_stacked.tif
Uploading /gdrive/MyDrive/ARD_ET/2018_Phillips/8/Working/LC08_L1TP_032032_20180901_20180912_01_T1_stacked.tif
Uploading 

In [ ]:
#@title create Google earth engine ARD Landsat8 ET ImageCollection
LALP_obj1.create_GEE_ARD_Landsat8_ET_ImageCollection(gee_ARD_path)

In [ ]:
#@title batch upload ARD Landsat8 LST images from google cloud storage to GEE
gcs_paths = LALP_obj1.list_gcs_bucket_files(gcs_bucket_name)
LALP_obj1.batch_upload_ARD_tiles_from_google_cloud_bucket(
    gcs_paths, 
    gee_ARD_path, 
    TBE_attribs, 
    attrib_name_sys_time='acquisition_date'
    )

gs://test-gee-upload-ard-et/2018_Phillips/8/LC08_L1TP_032032_20180104_20180118_01_T1_stacked.tif
gs://test-gee-upload-ard-et/2018_Phillips/8/LC08_L1TP_032032_20180410_20180417_01_T1_stacked.tif
gs://test-gee-upload-ard-et/2018_Phillips/8/LC08_L1TP_032032_20180426_20180502_01_T1_stacked.tif
gs://test-gee-upload-ard-et/2018_Phillips/8/LC08_L1TP_032032_20180613_20180703_01_T1_stacked.tif
gs://test-gee-upload-ard-et/2018_Phillips/8/LC08_L1TP_032032_20180629_20180716_01_T1_stacked.tif
gs://test-gee-upload-ard-et/2018_Phillips/8/LC08_L1TP_032032_20180715_20180730_01_T1_stacked.tif
gs://test-gee-upload-ard-et/2018_Phillips/8/LC08_L1TP_032032_20180731_20180814_01_T1_stacked.tif
gs://test-gee-upload-ard-et/2018_Phillips/8/LC08_L1TP_032032_20180816_20180829_01_T1_stacked.tif
gs://test-gee-upload-ard-et/2018_Phillips/8/LC08_L1TP_032032_20180901_20180912_01_T1_stacked.tif
gs://test-gee-upload-ard-et/2018_Phillips/8/LC08_L1TP_032032_20180917_20180928_01_T1_stacked.tif
gs://test-gee-upload-ard-et/20

## DEBUG

In [ ]:
TBE_attribs

In [ ]:
gee_ARD_path

'projects/ColoradoView/Landsat/ARD_ET/tmp'

In [ ]:
#gee_ARD_path
!earthengine create --help

Instructions for updating:
non-resource variables are not supported in the long term
usage: earthengine create [-h] {collection,folder} ...

Creates assets and folders.

optional arguments:
  -h, --help           show this help message and exit

Commands:
  {collection,folder}
    collection         Creates one or more image collections.
    folder             Creates one or more folders.


In [ ]:
test_asset_path = 'projects/ColoradoView/Landsat/ARD_tmp/tmp1'
cmd = ["earthengine", "create", "collection", "-p", test_asset_path] 
try:
    stdout = subprocess.check_output(cmd)
except subprocess.CalledProcessError as e:
    print(e.output)

In [ ]:
!tar --help

Usage: tar [OPTION...] [FILE]...
GNU 'tar' saves many files together into a single tape or disk archive, and can
restore individual files from the archive.

Examples:
  tar -cf archive.tar foo bar  # Create archive.tar from files foo and bar.
  tar -tvf archive.tar         # List all files in archive.tar verbosely.
  tar -xf archive.tar          # Extract all files from archive.tar.

 Local file name selection:

      --add-file=FILE        add given FILE to the archive (useful if its name
                             starts with a dash)
  -C, --directory=DIR        change to directory DIR
      --exclude=PATTERN      exclude files, given as a PATTERN
      --exclude-backups      exclude backup and lock files
      --exclude-caches       exclude contents of directories containing
                             CACHEDIR.TAG, except for the tag file itself
      --exclude-caches-all   exclude directories containing CACHEDIR.TAG
      --exclude-caches-under exclude everything under director

In [ ]:
#!ls {gDrv_ARD_RootPath}
!ls /gdrive/MyDrive/ColoradoView/ARD_ET/LC08_L1TP_033032_20180111_20180119_01_T1*

/gdrive/MyDrive/ColoradoView/ARD_ET/LC08_L1TP_033032_20180111_20180119_01_T1_eta.tif
/gdrive/MyDrive/ColoradoView/ARD_ET/LC08_L1TP_033032_20180111_20180119_01_T1_etf.tif
/gdrive/MyDrive/ColoradoView/ARD_ET/LC08_L1TP_033032_20180111_20180119_01_T1_etqa.tif
/gdrive/MyDrive/ColoradoView/ARD_ET/LC08_L1TP_033032_20180111_20180119_01_T1_etun.tif
/gdrive/MyDrive/ColoradoView/ARD_ET/LC08_L1TP_033032_20180111_20180119_01_T1_pctun.tif
/gdrive/MyDrive/ColoradoView/ARD_ET/LC08_L1TP_033032_20180111_20180119_01_T1_pixel_qa.tif
/gdrive/MyDrive/ColoradoView/ARD_ET/LC08_L1TP_033032_20180111_20180119_01_T1.xml


In [ ]:
!ls /gdrive/MyDrive/ColoradoView/ARD_ET/*.gz

/gdrive/MyDrive/ColoradoView/ARD_ET/LC080330322018011101T1-SC20210225173615.tar.gz
/gdrive/MyDrive/ColoradoView/ARD_ET/LC080330322018031601T1-SC20210225173445.tar.gz
/gdrive/MyDrive/ColoradoView/ARD_ET/LC080330322018040101T1-SC20210225173709.tar.gz
/gdrive/MyDrive/ColoradoView/ARD_ET/LC080330322018041701T1-SC20210225175520.tar.gz
/gdrive/MyDrive/ColoradoView/ARD_ET/LC080330322018060401T1-SC20210225181459.tar.gz
/gdrive/MyDrive/ColoradoView/ARD_ET/LC080330322018062001T1-SC20210225184139.tar.gz
/gdrive/MyDrive/ColoradoView/ARD_ET/LC080330322018070601T1-SC20210225181747.tar.gz
/gdrive/MyDrive/ColoradoView/ARD_ET/LC080330322018072201T1-SC20210225181940.tar.gz
/gdrive/MyDrive/ColoradoView/ARD_ET/LC080330322018080701T1-SC20210225175648.tar.gz
/gdrive/MyDrive/ColoradoView/ARD_ET/LC080330322018082301T1-SC20210225174825.tar.gz
/gdrive/MyDrive/ColoradoView/ARD_ET/LC080330322018090801T1-SC20210225173628.tar.gz
/gdrive/MyDrive/ColoradoView/ARD_ET/LC080330322018092401T1-SC20210225173639.tar.gz
/gdr

In [ ]:
Fst_FSN = !tar tzf /gdrive/MyDrive/ColoradoView/ARD_ET/Temp/LC080330322018011101T1-SC20210225173615.tar.gz | \
  awk -F/ '$NF ~ /\.tif$/ {print $NF ; exit }'
if (len(Fst_FSN) == 1 and ~str(Fst_FSN[0]).startswith('tar')):
  print(Fst_FSN[0])
  print(type(Fst_FSN[0]))

LC08_L1TP_033032_20180111_20180119_01_T1_pixel_qa.tif
<class 'str'>


In [ ]:
print(iFWN)
print(type(iFWN))
#Fst_FSN = !tar tzf /gdrive/MyDrive/ColoradoView/ARD_ET/Temp/LC080330322018011101T1-SC20210225173615.tar.gz \
%env CURRENT_FWN=$iFWN
Fst_FSN = !tar tzf $CURRENT_FWN \
  | awk -F/ '$NF ~ /\.tif$/ {print $NF ; exit }'
print(Fst_FSN)
if (len(Fst_FSN) == 1 and ~str(Fst_FSN[0]).startswith('tar')):
  print(Fst_FSN[0])
  print(type(Fst_FSN[0]))

/gdrive/MyDrive/ColoradoView/ARD_ET/Temp/LC080330322018011101T1-SC20210225173615.tar.gz
<class 'str'>
env: CURRENT_FWN=/gdrive/MyDrive/ColoradoView/ARD_ET/Temp/LC080330322018011101T1-SC20210225173615.tar.gz
['LC08_L1TP_033032_20180111_20180119_01_T1_pixel_qa.tif']
LC08_L1TP_033032_20180111_20180119_01_T1_pixel_qa.tif
<class 'str'>


In [ ]:
!ls /gdrive/MyDrive/ColoradoView/ARD_ET/Temp

'Copy of LC080330322018011101T1-SC20210225173615.tar._gz_'
'Copy of LC080330322018031601T1-SC20210225173445.tar._gz_'
 LC080330322018011101T1-SC20210225173615.tar.gz
 LC080330322018031601T1-SC20210225173445.tar.gz


In [ ]:
!cat /gdrive/MyDrive/ColoradoView/ARD_ET/LC08_L1TP_033032_20180111_20180119_01_T1.xml

In [ ]:
import os
import re
import subprocess
from osgeo import gdal
import xml.etree.ElementTree as ET
import datetime
import platform
import copy

from urllib.parse import urlparse


In [ ]:
def glob_re(pattern, strings):
    return list(filter(re.compile(pattern).search, strings))

In [ ]:
#!tar --help
#!ls {gDrv_ARD_RootPath}
ARDTileSearchText = r'LC.*T1' #r'C\d\d_V\d\d'
TgtLayers = ['eta', 'pixel_qa', 'etun', 'etf']
ExcludeLayers = ['etqa', 'pctun']
DataRootFolder = '/gdrive/MyDrive/ColoradoView/ARD_ET/Temp'
FP = DataRootFolder
FSNs_found = glob_re(ARDTileSearchText + r'.*\.gz', os.listdir(FP))
#print(FSNs_found)
FSNs_found = sorted(FSNs_found) # acsd.
# get the first tif file name in the gz file and extract the Prefix string
ARDTilePrefixArr = []
for ii, iFSN in enumerate(FSNs_found):
  iFWN = os.path.join(DataRootFolder, iFSN)
  %env CURRENT_FWN=$iFWN
  fst_tif_FSN = !tar tzf $CURRENT_FWN | \
    awk -F/ '$NF ~ /\.tif$/ {print $NF ; exit }'
  #print(fst_tif_FSN)
  if (len(fst_tif_FSN) == 1 and ~str(fst_tif_FSN[0]).startswith('tar')):
    i_prefix = re.split(r'(^.*_T1'+r')', fst_tif_FSN[0])[1]
    #print(i_prefix)
    ARDTilePrefixArr.append(i_prefix)
  else:
    raise ValueError(
      'Error: ' + iFWN + ' doesn\'t contain tif file(s). Cannot tar!')

## debug ##
for iPrefix, iFSN in zip(ARDTilePrefixArr, FSNs_found):
    print(iPrefix, iFSN)

env: CURRENT_FWN=/gdrive/MyDrive/ColoradoView/ARD_ET/Temp/LC080330322018011101T1-SC20210225173615.tar.gz
LC08_L1TP_033032_20180111_20180119_01_T1
env: CURRENT_FWN=/gdrive/MyDrive/ColoradoView/ARD_ET/Temp/LC080330322018031601T1-SC20210225173445.tar.gz
LC08_L1TP_033032_20180316_20180402_01_T1
LC08_L1TP_033032_20180111_20180119_01_T1 LC080330322018011101T1-SC20210225173615.tar.gz
LC08_L1TP_033032_20180316_20180402_01_T1 LC080330322018031601T1-SC20210225173445.tar.gz


In [ ]:
# !mkdir -p /gdrive/MyDrive/ColoradoView/ARD_ET/Temp/RawTif
# !tar -xvf /gdrive/MyDrive/ColoradoView/ARD_ET/Temp/LC080330322018011101T1-SC20210225173615.tar.gz \
#      -C /gdrive/MyDrive/ColoradoView/ARD_ET/Temp/RawTif \
#      --exclude='*_etqa.tif' --exclude='*_pctun.tif'

LC08_L1TP_033032_20180111_20180119_01_T1_pixel_qa.tif
LC08_L1TP_033032_20180111_20180119_01_T1_etf.tif
LC08_L1TP_033032_20180111_20180119_01_T1_eta.tif
LC08_L1TP_033032_20180111_20180119_01_T1_etun.tif
LC08_L1TP_033032_20180111_20180119_01_T1.xml


In [ ]:
#!rm -R /gdrive/MyDrive/ColoradoView/ARD_ET/Temp/RawTif/*
!ls -la /gdrive/MyDrive/ColoradoView/ARD_ET/Temp/RawTif

total 0


In [ ]:

#for iATP in self.ARDTilePrefixArr
# res_findTgtLyrs = self.extract_target_layers(
#                 iATP, 
#                 iATP + "_ST.tar", 
#                 SW_extract_xml = True
#                 )

exclude_args_arr = []
for iExcludeLyr in ExcludeLayers:
  #exclude_args_arr.append('--exclude=\'*_' + iExcludeLyr + '.tif\'')
  exclude_args_arr.append('--exclude')
  exclude_args_arr.append('*_' + iExcludeLyr + '.tif')

print(exclude_args_arr)

#file_prefix = ARDTilePrefixArr[0]
cur_ST_tar_FSN = FSNs_found[0]

output_folder = os.path.join(DataRootFolder, 'RawTif')
print(output_folder)

cmd_str_pieces = ["tar", "-xvf", 
     os.path.join(DataRootFolder, cur_ST_tar_FSN),
     "-C", 
     output_folder,
    ]
cmd_str_pieces.extend(exclude_args_arr)
print(cmd_str_pieces)

stdout = subprocess.check_output(
    cmd_str_pieces#,
    #cwd=DataWorkingFolder
)

extracted_FSNs = stdout.decode("utf-8").split(sep="\n")


['--exclude', '*_etqa.tif', '--exclude', '*_pctun.tif']
/gdrive/MyDrive/ColoradoView/ARD_ET/Temp/RawTif
['tar', '-xvf', '/gdrive/MyDrive/ColoradoView/ARD_ET/Temp/LC080330322018011101T1-SC20210225173615.tar.gz', '-C', '/gdrive/MyDrive/ColoradoView/ARD_ET/Temp/RawTif', '--exclude', '*_etqa.tif', '--exclude', '*_pctun.tif']


In [ ]:
#print(type(stdout))
extracted_FSNs = stdout.decode("utf-8").split(sep="\n")
print(type(extracted_FSNs))
print(extracted_FSNs)

#extracted_FSNs = str(stdout).split(sep="\n")
#print(extracted_FSNs)

<class 'list'>
['LC08_L1TP_033032_20180111_20180119_01_T1_pixel_qa.tif', 'LC08_L1TP_033032_20180111_20180119_01_T1_etf.tif', 'LC08_L1TP_033032_20180111_20180119_01_T1_eta.tif', 'LC08_L1TP_033032_20180111_20180119_01_T1_etun.tif', 'LC08_L1TP_033032_20180111_20180119_01_T1.xml', '']


In [ ]:
!ls -la /gdrive/MyDrive/ColoradoView/ARD_ET/Temp/RawTif

total 471340
-rw------- 1 root root 120660616 Feb 25 17:36 LC08_L1TP_033032_20180111_20180119_01_T1_eta.tif
-rw------- 1 root root 120660620 Feb 25 17:36 LC08_L1TP_033032_20180111_20180119_01_T1_etf.tif
-rw------- 1 root root 120660668 Feb 25 17:36 LC08_L1TP_033032_20180111_20180119_01_T1_etun.tif
-rw------- 1 root root 120660612 Feb 25 17:36 LC08_L1TP_033032_20180111_20180119_01_T1_pixel_qa.tif
-rw------- 1 root root      7785 Feb 25 17:36 LC08_L1TP_033032_20180111_20180119_01_T1.xml


In [ ]:
!gdalinfo --help

Usage: gdalinfo [--help-general] [-json] [-mm] [-stats] [-hist] [-nogcp] [-nomd]
                [-norat] [-noct] [-nofl] [-checksum] [-proj4]
                [-listmdd] [-mdd domain|`all`]*
                [-sd subdataset] [-oo NAME=VALUE]* datasetname


In [ ]:
 
tmpFSNs = glob_re(r'.*\.tif', os.listdir(DataRootFolder + '/RawTif/'))
for iFSN in tmpFSNs:
  !gdalinfo {os.path.join(DataRootFolder, 'RawTif', iFSN)}


In [ ]:
print(iFWN)

/gdrive/MyDrive/ColoradoView/ARD_ET/Temp/LC080330322018011101T1-SC20210225173615.tar.gz


In [ ]:
class AttrDict(dict):
    __getattr__ = dict.__getitem__
    __setattr__ = dict.__setitem__

In [ ]:
def parse_one_ARD_tile_xml_global_attributes(xml_FWN, 
                          target_tag_arr, 
                          gee_attrib_names,
                          name_space_dict = {'ns0': 'http://espa.cr.usgs.gov/v2'} #{'ns0': 'https://landsat.usgs.gov/ard/v1'} 
                        ):
  
  ret_dict = AttrDict()

  cur_xml_tree = ET.parse(xml_FWN)
  cur_xml_root = cur_xml_tree.getroot()
  
  ns = name_space_dict
  
  for iTag, iGEEName in zip(target_tag_arr, gee_attrib_names):
      #("ns0:tile_metadata/"
      # "ns0:global_metadata/"
      # "ns0:projection_information")
      iTagInfo = cur_xml_root.findall(iTag, ns)
      
      ## main field text
      if (iTagInfo[0].text is not None):
          iTagInfo0_text = iTagInfo[0].text
          iTagInfo0_text = re.sub("\s", '', iTagInfo0_text)
          if (len(iTagInfo0_text) > 0):
              #print(iGEEName, iTagInfo[0].text)
              ret_dict[iGEEName] = iTagInfo[0].text
      
      ## attributes
      ###
      #<bounding_coordinates>
      #    <west>-110.131907577</west>
      #    <east>-108.18643133</east>
      #    <north>38.7343536882</north>
      #    <south>37.2242306639</south>
      #</bounding_coordinates>
      if (len(iTagInfo[0]) > 0):
          # sub nodes 
          for iSubE in list(iTagInfo[0].iter()):
              Text_iSubE = iSubE.text
              Text_iSubE = re.sub("\s", '', Text_iSubE)
              TagName_iSubE = iSubE.tag
              TagName_iSubE = re.sub("{" + name_space_dict.get(list(name_space_dict.keys())[0]) + "}",
                                  '',
                                  TagName_iSubE
                                  )
              if (len(Text_iSubE) > 0):
                  iSubE_GEEName = iGEEName + '_' + TagName_iSubE
                  #print(iSubE_GEEName, Text_iSubE)
                  ret_dict[iSubE_GEEName] = Text_iSubE

      ###
      # <tile_grid h="009" v="010"/>
      iTag_items = iTagInfo[0].items()
      if (len(iTag_items) > 0):
          for jIt in iTag_items:
              jIt_GEEName = iGEEName + '_' + jIt[0]
              #print(jIt_GEEName, jIt[1])
              ret_dict[jIt_GEEName] = jIt[1]
              
  return ret_dict


In [ ]:
def parse_one_ARD_tile_xml_one_band_attributes(
    xml_FWN, 
    target_band_path, #"ns0:bands/"
    target_band_name, #"eta"
    target_band_attribs, # ['add_offset', 'data_type', 'name', 'valid_range']
    name_space_dict = {ns: 'http://espa.cr.usgs.gov/v2'} #{'ns0': 'https://landsat.usgs.gov/ard/v1'} 
    ):
    
    def search_matched_attrib_name(attrib_name, TBC_attrib_name_arr):
        try:
            idx = TBC_attrib_name_arr.index(attrib_name)
        except:
            idx = -1
        return idx
    
    ret_dict = AttrDict()

    cur_xml_tree = ET.parse(xml_FWN)
    cur_xml_root = cur_xml_tree.getroot()
    
    ns = name_space_dict
    
    ## find the node of the target band
    #("ns0:bands/")
    TagInfo_arr = cur_xml_root.findall(target_band_path, ns)
    for iTagInfo in TagInfo_arr:
        ## find target band by "name" 
        # <band product="evapotranspiration" source="toa_refl" name="eta" 
        # category="image" data_type="INT16" nlines="7831" nsamps="7701" 
        # fill_value="-9999" scale_factor="0.001000">
        iTag_items = iTagInfo.items()
        if (len(iTag_items) > 0):
            Is_iTag_target = False
            for jIt in iTag_items:
                if (jIt[0] == 'name' and jIt[1] == target_band_name):
                    Is_iTag_target = True
                    break
            if (Is_iTag_target == False):
                # not the target band, searching the next band
                #print("not the target band, searching the next band")
                continue
            
            for jIt in iTag_items:
                jIt_idx = search_matched_attrib_name(jIt[0], target_band_attribs)
                if (jIt_idx >= 0):
                    jIt_GEEName = target_band_name + '_' + jIt[0]
                    print(jIt_GEEName, jIt[1])
                    ret_dict[jIt_GEEName] = jIt[1]
                    del target_band_attribs[jIt_idx]
                    
        print("{} band attribs in sub elements:".format(target_band_name))
        
        if (len(iTagInfo) > 0):
            # sub nodes 
            for iSubE in list(iTagInfo.iter()):

                TagName_iSubE = iSubE.tag
                TagName_iSubE = re.sub("{" + name_space_dict.get(list(name_space_dict.keys())[0]) + "}",
                                    '',
                                    TagName_iSubE
                                    )
                TagName_iSubE_idx = search_matched_attrib_name(TagName_iSubE, target_band_attribs)
                if (TagName_iSubE_idx < 0):
                    continue
                #print("TagName_iSubE", TagName_iSubE, isinstance(iSubE, str), type(iSubE))
                
                #if (not isinstance(iSubE, str)):
                ##            
                # <valid_range min="0.000000" max="20000.000000"/>
                # <pixel_size x="30" y="30" units="meters"/>
                iSubE_items = iSubE.items()
                if (len(iSubE_items) > 0):
                    for jIt in iSubE_items:
                        jIt_GEEName = target_band_name + '_' + TagName_iSubE + '_' + jIt[0]
                        #print(jIt_GEEName, jIt[1])
                        ret_dict[jIt_GEEName] = jIt[1]
                else:
                    ##
                    # <data_units>mm</data_units>
                    Text_iSubE = iSubE.text
                    Text_iSubE = re.sub("\s", '', Text_iSubE)
                    if (len(Text_iSubE) > 0):
                        iSubE_GEEName = target_band_name + '_' + TagName_iSubE
                        #print(iSubE_GEEName, Text_iSubE)
                        ret_dict[iSubE_GEEName] = Text_iSubE
                        
                del target_band_attribs[TagName_iSubE_idx]
                            
              
        # break from band loop
        break
        
    if (len(target_band_attribs) > 0):
        print("remaining target_band_attribs", target_band_attribs)
        
    return ret_dict

In [ ]:
## test xml parsing
#DataRootFolder = '/gdrive/MyDrive/ColoradoView/ARD_ET/Temp'
#test_xml_FWN = os.path.join(DataRootFolder, 'Working', 'LC08_CU_013011_20181231_20190201_C01_V01.xml')
test_xml_FWN = os.path.join(DataRootFolder, 'Working', 'LC08_L1TP_033032_20180111_20180119_01_T1.xml')

ns = 'ns0'
name_space_dict = {ns: 'http://espa.cr.usgs.gov/v2'} #{ns: 'https://landsat.usgs.gov/ard/v1'}

#### global attributes ####
tag_str_prefix_tile_global = "{}:global_metadata/".format(ns) #"{}:tile_metadata/{}:global_metadata/".format(ns, ns)
global_tag_strs = TBE_attribs['global']
global_tag_xml_path_arr = [
    "{}{}:{}".format(tag_str_prefix_tile_global, ns, iTagStr) for iTagStr in global_tag_strs
    ]        
global_gee_attrib_names = global_tag_strs

print(global_tag_strs)
print(global_tag_xml_path_arr)
print(global_gee_attrib_names)

ret_dict_glb = parse_one_ARD_tile_xml_global_attributes(test_xml_FWN, 
    global_tag_xml_path_arr, 
    global_gee_attrib_names,
    name_space_dict = name_space_dict)
#combined_attr_dict = LALP_obj1.parse_one_ARD_xml(test_xml_FWN, TBE_attribs, attrib_name_sys_time='acquisition_date')


['acquisition_date', 'satellite', 'instrument', 'product_id', 'bounding_coordinates']
['ns0:global_metadata/ns0:acquisition_date', 'ns0:global_metadata/ns0:satellite', 'ns0:global_metadata/ns0:instrument', 'ns0:global_metadata/ns0:product_id', 'ns0:global_metadata/ns0:bounding_coordinates']
['acquisition_date', 'satellite', 'instrument', 'product_id', 'bounding_coordinates']


In [ ]:
import pprint
pprint.pprint(ret_dict_glb)

{'acquisition_date': '2018-01-11',
 'bounding_coordinates_east': '-102.853554',
 'bounding_coordinates_north': '41.388764',
 'bounding_coordinates_south': '39.253625',
 'bounding_coordinates_west': '-105.616151',
 'instrument': 'OLI_TIRS',
 'product_id': 'LC08_L1TP_033032_20180111_20180119_01_T1',
 'satellite': 'LANDSAT_8'}


In [ ]:
#TBE_attribs

In [ ]:
import copy
#### band attributes ####
TgtLayers_attribs = copy.deepcopy(TBE_attribs['bands'])
TgtLayers_attribs_keys = list(TgtLayers_attribs.keys())

tag_str_prefix_tile_band = "{}:bands/".format(ns)

#for iBand in TgtLayers_attribs_keys:
iBand = TgtLayers_attribs_keys[3]
print("processing band [{}]".format(iBand))
            
target_band_path_iBand = tag_str_prefix_tile_band
target_band_name_iBand = iBand
#["name", "scale_factor", "add_offset", "data_type", "fill_value", "nlines", "nsamps",
# "pixel_size", "data_units", "valid_range", "app_version"]
target_band_attribs_iBand = TgtLayers_attribs[iBand]
attr_dict_iBand = parse_one_ARD_tile_xml_one_band_attributes(
    test_xml_FWN, 
    target_band_path_iBand, #"ns0:bands/"
    target_band_name_iBand, #"eta"
    target_band_attribs_iBand, # ['add_offset', 'data_type', 'name', 'valid_range']
    name_space_dict = name_space_dict #{'ns0': 'https://landsat.usgs.gov/ard/v1'} 
)
import pprint
pprint.pprint(attr_dict_iBand)


processing band [etf]
etf_name etf
etf_data_type INT16
etf_nlines 7831
etf_nsamps 7701
etf_fill_value -9999
etf_scale_factor 0.000100
etf band attribs in sub elements:
{'etf_app_version': 'et_2.0.0(Collection1)',
 'etf_data_type': 'INT16',
 'etf_data_units': 'unitless',
 'etf_fill_value': '-9999',
 'etf_long_name': 'Evapotranspirationfraction',
 'etf_name': 'etf',
 'etf_nlines': '7831',
 'etf_nsamps': '7701',
 'etf_pixel_size_units': 'meters',
 'etf_pixel_size_x': '30',
 'etf_pixel_size_y': '30',
 'etf_scale_factor': '0.000100',
 'etf_valid_range_max': '10500.000000',
 'etf_valid_range_min': '0.000000'}


In [ ]:
## debug
import subprocess
GCS_FWN = 'gs://test-gee-upload-ard-lst/2000/test1/LT05_CU_009008_20000108_20190116_C01_V01_stacked.tif'
Full_asset_name = 'projects/ColoradoView/Landsat/ARD/test1/LT05_CU_009008_20000108_20190116_C01_V01_stacked'
property_op_str_arr = [
#"-p(date)time_start=2000-01-08",
'--nodata_value=-9999']
#options_others = ['--time_start=2000-01-08T00:00:00']
#'gs://test-gee-upload-ard-lst/2000/test1/LT05_CU_009008_20000106_20190523_C01_V01_stacked.tif']
time_start_str = '--time_start=2000-01-08'

In [ ]:
#"earthengine upload image --asset_id=users/stevenf/cms/rehfeldt_1km/" + name + 
#"  --pyramiding_policy=mean --nodata_value -500 " + path
cmd = ["earthengine", "upload", "image"]
cmd.extend(["--asset_id={}".format(Full_asset_name)])
cmd.extend(property_op_str_arr)
#cmd.extend(options_others)
cmd.extend([time_start_str])
cmd.extend([GCS_FWN])
print(cmd)
try:
    stdout = subprocess.check_output(cmd)
    print('here')
    #pass
    
except subprocess.CalledProcessError as e:
    print(e.output)
                


['earthengine', 'upload', 'image', '--asset_id=projects/ColoradoView/Landsat/ARD/test1/LT05_CU_009008_20000108_20190116_C01_V01_stacked', '--nodata_value=-9999', '--time_start=2000-01-08', 'gs://test-gee-upload-ard-lst/2000/test1/LT05_CU_009008_20000108_20190116_C01_V01_stacked.tif']
here


In [ ]:
!earthengine upload image --help

usage: earthengine upload image [-h] [--wait [WAIT]] [--force]
                                [--asset_id ASSET_ID] [--last_band_alpha]
                                [--nodata_value NODATA_VALUE]
                                [--pyramiding_policy PYRAMIDING_POLICY]
                                [--bands BANDS] [--crs CRS]
                                [--manifest MANIFEST] [--property PROPERTY]
                                [--time_start TIME_START]
                                [--time_end TIME_END]
                                [src_files [src_files ...]]

Uploads an image from Cloud Storage to Earth Engine. See docs for "asset set"
for additional details on how to specify asset metadata properties.

positional arguments:
  src_files             Cloud Storage URL(s) of the file(s) to upload. Must
                        have the prefix 'gs://'.

optional arguments:
  -h, --help            show this help message and exit
  --wait [WAIT], -w [WAIT]
                      